In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

/Users/jeanlima/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/jeanlima/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
classifier = Sequential()
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'tanh'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(64, (3, 3),  activation = 'tanh'))
classifier.add(Convolution2D(64, (3, 3),  activation = 'tanh'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Convolution2D(128, (3, 3),  activation = 'tanh'))
classifier.add(Convolution2D(128, (3, 3),  activation = 'tanh'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(activation = 'relu', units = 512))
classifier.add(Dropout(0.2))
classifier.add(Dense(activation = 'relu', units = 256))
classifier.add(Dropout(0.2))
classifier.add(Dense(activation = 'softmax', units = 15))

In [10]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
training_set = train_datagen.flow_from_directory(
        '../CreatingDeepLearningDataset/TimeDomain/Dataset-Train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        '../CreatingDeepLearningDataset/TimeDomain/Dataset-Test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 2699 images belonging to 15 classes.
Found 301 images belonging to 15 classes.


In [13]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=2699//32,
        epochs=30,
        validation_data=test_set,
        validation_steps=301//32)

Epoch 1/30
84/84 [==============================] - 49s 584ms/step - loss: 2.2647 - acc: 0.1831 - val_loss: 1.5725 - val_acc: 0.3355
Epoch 2/30
84/84 [==============================] - 45s 536ms/step - loss: 1.2895 - acc: 0.4842 - val_loss: 1.0540 - val_acc: 0.4850
Epoch 3/30
84/84 [==============================] - 45s 535ms/step - loss: 0.9492 - acc: 0.5864 - val_loss: 0.7114 - val_acc: 0.6811
Epoch 4/30
84/84 [==============================] - 48s 569ms/step - loss: 0.7828 - acc: 0.6644 - val_loss: 0.6658 - val_acc: 0.6611
Epoch 5/30
84/84 [==============================] - 44s 522ms/step - loss: 0.7315 - acc: 0.6824 - val_loss: 0.8176 - val_acc: 0.6080
Epoch 6/30
84/84 [==============================] - 44s 529ms/step - loss: 0.6849 - acc: 0.7122 - val_loss: 0.5378 - val_acc: 0.7741
Epoch 7/30
84/84 [==============================] - 45s 531ms/step - loss: 0.6193 - acc: 0.7319 - val_loss: 0.4888 - val_acc: 0.8040
Epoch 8/30
84/84 [==============================] - 45s 537ms/step - 

In [14]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=2699//32,
        epochs=5,
        validation_data=test_set,
        validation_steps=301//32)

Epoch 1/5
84/84 [==============================] - 41s 493ms/step - loss: 0.3050 - acc: 0.8702 - val_loss: 0.3889 - val_acc: 0.8472
Epoch 2/5
84/84 [==============================] - 45s 535ms/step - loss: 0.3190 - acc: 0.8743 - val_loss: 0.4082 - val_acc: 0.8372
Epoch 3/5
84/84 [==============================] - 44s 526ms/step - loss: 0.3004 - acc: 0.8740 - val_loss: 0.4638 - val_acc: 0.8206
Epoch 4/5
84/84 [==============================] - 46s 552ms/step - loss: 0.2854 - acc: 0.8784 - val_loss: 0.2423 - val_acc: 0.8671
Epoch 5/5
84/84 [==============================] - 44s 527ms/step - loss: 0.2629 - acc: 0.8903 - val_loss: 0.3529 - val_acc: 0.8339


In [15]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=2699//32,
        epochs=5,
        validation_data=test_set,
        validation_steps=301//32)

Epoch 1/5
84/84 [==============================] - 44s 519ms/step - loss: 0.3164 - acc: 0.8702 - val_loss: 0.4689 - val_acc: 0.8173
Epoch 2/5
84/84 [==============================] - 44s 520ms/step - loss: 0.3177 - acc: 0.8750 - val_loss: 0.2629 - val_acc: 0.9037
Epoch 3/5
84/84 [==============================] - 44s 524ms/step - loss: 0.3292 - acc: 0.8698 - val_loss: 0.3461 - val_acc: 0.8538
Epoch 4/5
84/84 [==============================] - 44s 529ms/step - loss: 0.2826 - acc: 0.8884 - val_loss: 0.4939 - val_acc: 0.8206
Epoch 5/5
84/84 [==============================] - 44s 523ms/step - loss: 0.2795 - acc: 0.8854 - val_loss: 0.3178 - val_acc: 0.8837


In [20]:
import pandas as pd
import numpy as np
predict_gen = test_datagen.flow_from_directory(directory='../CreatingDeepLearningDataset/TimeDomain/Dataset-Test',
                                                              target_size=[64,64],
                                                              batch_size=4123,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = classifier.predict(X_val_sample)
#classifier.evaluate_generator(test_datagen, steps=1, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(onedtrue, onedpred))
from sklearn.metrics import classification_report
target_names = ['Geladeira', 'Liquidificador', 'LiquidificadorGeladeira',
               'Notebook', 'NotebookGeladeira', 'NotebookLiquidificador',
               'NotebookLiquidificadorGeladeira', 'NotebookVentilador',
               'NotebookVentiladorGeladeira', 'NotebookVentiladorLiquidificador',
               'NotebookVentiladorLiquidificadorGeladeira', 'Ventilador',
               'VentiladorGeladeira', 'VentiladorLiquidificador',
               'VentiladorLiquidificadorGeladeira']

print(classification_report(onedtrue, onedpred, target_names=target_names))

Found 301 images belonging to 15 classes.
[[17  0  0  0  0  0  0  0  0  0  0  0  3  0  0]
 [ 0 19  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0 15  0  0  0  1  0  0  0  0  0  0  0  4]
 [ 0  0  0 19  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0 16  0  0  0  4  0  0  0  1  0]
 [ 0  0  0  0  0  0 18  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 20  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0 19  0  0  0  0  0]
 [ 0  0  0  0  0  0  5  0  0  0 14  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0 20  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 20  0  0]
 [ 0  1  0  0  0  1  0  0  0  0  0  0  0 18  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0 17]]
                                           precision    recall  f1-score   support

                                Geladeira       1.00      0.85      0.92        20
                           Liquidificador       0.95      0.95      0.

In [21]:
classifier.save('timedomain-88-basic.h5')